In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, Input
from tensorflow.keras.datasets import cifar100
import matplotlib.pyplot as plt

In [ ]:
# Load and preprocess the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values
y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [ ]:
# Define the CNN model
cnn_model = Sequential([
    # Add preprocessing layers
    resize_and_rescale,
    data_augmentation,

    # Input layer
    Input(shape=(32, 32, 3)),

    # Block 1
    layers.Conv2D(16, (3, 3), padding='same', activation=None),
    layers.LeakyReLU(negative_slope=0.1),
    layers.MaxPooling2D(),

    # Block 2
    layers.Conv2D(64, (3, 3), activation=None, padding='same'),
    layers.LeakyReLU(negative_slope=0.1),
    layers.Conv2D(64, (3, 3), activation=None, padding='same'),
    layers.LeakyReLU(negative_slope=0.1),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Block 3
    layers.Conv2D(128, (3, 3), activation=None, padding='same'),
    layers.LeakyReLU(negative_slope=0.1),
    layers.Conv2D(128, (3, 3), activation=None, padding='same'),
    layers.LeakyReLU(negative_slope=0.1),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Block 4
    layers.Conv2D(256, (3, 3), activation=None, padding='same'),
    layers.LeakyReLU(negative_slope=0.1),
    layers.Conv2D(256, (3, 3), activation=None, padding='same'),
    layers.LeakyReLU(negative_slope=0.1),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    # Fully connected layers
    layers.Flatten(),
    layers.Dense(512, activation=None),
    layers.LeakyReLU(negative_slope=0.1),
    layers.Dropout(0.5),
    layers.Dense(100, activation='softmax')  # Assuming 100 classes
])

cnn_model.summary()

In [ ]:
# Compile the model
cnn_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
# Train the model
history = cnn_model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test))

Epoch 1/50
 30/782 ━━━━━━━━━━━━━━━━━━━━ 18:21 1s/step - accuracy: 0.0075 - loss: 4.6126

In [ ]:
# Evaluate the model
test_loss, test_acc = cnn_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()